#### Import Libraries

In [2]:
# pip install segmentation-models

In [3]:
# pip install -q git+https://github.com/tensorflow/examples.git

In [4]:
# pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [5]:
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import math

# import detectron2
# import segmentation_models as sm

from sklearn.model_selection import train_test_split
from sklearn import metrics

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import plot_model

from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPooling2D, Dropout, Conv2DTranspose, concatenate, ZeroPadding2D, Concatenate
from tensorflow.keras import Model, Input
from keras import backend as K

import cv2

#### Data Preparation

In [6]:
# Mapping color from kaggle (in BGR format)
# Building: #3C1098 --> Use building only = [152, 16, 60]
# Land (unpaved area): #8429F6
# Road: #6EC1E4
# Vegetation: #FEDD3A
# Water: #E2A929
# Unlabeled: #9B9B9B

# Load data
dataset_feature = []
dataset_label = []
dataset_label2 = []

for i in range(1, 9):
  for j in range(1, 37):
    if j < 10:
        path = "../input/aerial-building-only-augmented/Semantic segmentation dataset augmented/Tile " + str(i) + "/images/image_part_00" + str(j) + ".jpg"
    else:
        path = "../input/aerial-building-only-augmented/Semantic segmentation dataset augmented/Tile " + str(i) + "/images/image_part_0" + str(j) + ".jpg"
    
    img = cv2.imread(path)
    img2 = cv2.resize(img, (512, 512), interpolation=cv2.INTER_NEAREST)

    dataset_feature.append(img2)

dataset_feature = np.array(dataset_feature)
dataset_feature = dataset_feature.astype('float32')

cnt0 = 0
cnt1 = 1

for i in range(1, 9):
  for j in range(1, 37):
    if j < 10:
        path = "../input/aerial-building-only-augmented/Semantic segmentation dataset augmented/Tile " + str(i) + "/masks/image_part_00" + str(j) + ".png"
    else:
        path = "../input/aerial-building-only-augmented/Semantic segmentation dataset augmented/Tile " + str(i) + "/masks/image_part_0" + str(j) + ".png"
    
    img = cv2.imread(path)
    img2 = cv2.resize(img, (512, 512), interpolation=cv2.INTER_NEAREST)
    
    dataset_label.append(img2)

    # Convert the color value
    temp = []
    for ii in range(512):
      temp2 = []
      for jj in range(512):
        if img2[ii][jj].tolist() == [152, 16, 60]:
          temp2.append(1)
          cnt1 += 1
        else:
          temp2.append(0)
          cnt0 += 1
      temp.append(temp2)
    dataset_label2.append(temp)

print("Counter Label 0:", cnt0)
print("Counter Label 1:", cnt1)

# dataset_label = np.array(dataset_label)
dataset_label2 = np.array(dataset_label2)
dataset_label2 = dataset_label2.astype('float32')
# dataset_label = dataset_label / 255.0
# dataset_label = dataset_label.astype('float32')

x_train_val, x_testing, y_train_val, y_testing = train_test_split(dataset_feature, dataset_label2)
x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val)
print(x_train_val.shape)
print(x_testing.shape)
print(y_train_val.shape)
print(y_testing.shape)

Counter Label 0: 67553001
Counter Label 1: 7944472
(216, 512, 512, 3)
(72, 512, 512, 3)
(216, 512, 512)
(72, 512, 512)


In [7]:
arr_label = []
image_label = []

sz = x_testing.shape[0]
for i in range(sz):
  temp_label = []
  temp_image_label = []
  for j in range(512):
    temp_label2 = []
    temp_image_label2 = []
    for k in range(512):
      label = y_testing[i][j][k]
      if label == 1:
        label_image = [0, 0, 0]
      else:
        label_image = [255, 255, 255]
      temp_label2.append(label)
      temp_image_label2.append(label_image)
    temp_label.append(temp_label2)
    temp_image_label.append(temp_image_label2)
  arr_label.append(temp_label)
  image_label.append(temp_image_label)

arr_label = np.array(arr_label)
image_label = np.array(image_label)

#### Metric Evaluation and Loss Function

In [8]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# Class Weight Loss for Imbalance Dataset
def create_weighted_binary_crossentropy(zero_weight, one_weight):
    def weighted_binary_crossentropy(y_true, y_pred):
        # Calculate the binary crossentropy
        b_ce = K.binary_crossentropy(y_true, y_pred)

        # Apply the weights
        weight_vector = y_true * one_weight + (1.0 - y_true) * zero_weight
        weighted_b_ce = weight_vector * b_ce

        # Return the mean error
        return K.mean(weighted_b_ce)

    return weighted_binary_crossentropy

#### UNET Model

In [9]:
def conv2d_block(input_tensor, n_filters, kernel_size = 3, batchnorm = True):
    """Function to add 2 convolutional layers with the parameters passed to it"""
    # first layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # second layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    return x
  
def get_unet(input_img, n_filters = 16, dropout = 0.1, batchnorm = True):
    # Contracting Path
    c1 = conv2d_block(input_img, n_filters * 1, kernel_size = 3, batchnorm = batchnorm)
    p1 = MaxPooling2D((2, 2))(c1)
    p1 = Dropout(dropout)(p1)
    
    c2 = conv2d_block(p1, n_filters * 2, kernel_size = 3, batchnorm = batchnorm)
    p2 = MaxPooling2D((2, 2))(c2)
    p2 = Dropout(dropout)(p2)
    
    c3 = conv2d_block(p2, n_filters * 4, kernel_size = 3, batchnorm = batchnorm)
    p3 = MaxPooling2D((2, 2))(c3)
    p3 = Dropout(dropout)(p3)
    
    c4 = conv2d_block(p3, n_filters * 8, kernel_size = 3, batchnorm = batchnorm)
    p4 = MaxPooling2D((2, 2))(c4)
    p4 = Dropout(dropout)(p4)
    
    c5 = conv2d_block(p4, n_filters = n_filters * 16, kernel_size = 3, batchnorm = batchnorm)
    
    # Expansive Path
    u6 = Conv2DTranspose(n_filters * 8, (3, 3), strides = (2, 2), padding = 'same')(c5)
    u6 = concatenate([u6, c4])
    u6 = Dropout(dropout)(u6)
    c6 = conv2d_block(u6, n_filters * 8, kernel_size = 3, batchnorm = batchnorm)
    
    u7 = Conv2DTranspose(n_filters * 4, (3, 3), strides = (2, 2), padding = 'same')(c6)
    u7 = concatenate([u7, c3])
    u7 = Dropout(dropout)(u7)
    c7 = conv2d_block(u7, n_filters * 4, kernel_size = 3, batchnorm = batchnorm)
    
    u8 = Conv2DTranspose(n_filters * 2, (3, 3), strides = (2, 2), padding = 'same')(c7)
    u8 = concatenate([u8, c2])
    u8 = Dropout(dropout)(u8)
    c8 = conv2d_block(u8, n_filters * 2, kernel_size = 3, batchnorm = batchnorm)
    
    u9 = Conv2DTranspose(n_filters * 1, (3, 3), strides = (2, 2), padding = 'same')(c8)
    u9 = concatenate([u9, c1])
    u9 = Dropout(dropout)(u9)
    c9 = conv2d_block(u9, n_filters * 1, kernel_size = 3, batchnorm = batchnorm)
    
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)
    model = Model(inputs=[input_img], outputs=[outputs])
    return model

#### InceptionResNetV2-UNet Model (Transfer Learning)

In [10]:
from tensorflow.keras.applications import InceptionResNetV2

def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_inception_resnetv2_unet(input_shape):
    """ Input """
    inputs = Input(input_shape)

    """ Pre-trained InceptionResNetV2 Model """
    encoder = InceptionResNetV2(include_top=False, weights="imagenet", input_tensor=inputs)

    """ Encoder """
    s1 = encoder.get_layer("input_1").output           ## (512 x 512)

    s2 = encoder.get_layer("activation").output        ## (255 x 255)
    s2 = ZeroPadding2D(( (1, 0), (1, 0) ))(s2)         ## (256 x 256)

    s3 = encoder.get_layer("activation_3").output      ## (126 x 126)
    s3 = ZeroPadding2D((1, 1))(s3)                     ## (128 x 128)

    s4 = encoder.get_layer("activation_74").output      ## (61 x 61)
    s4 = ZeroPadding2D(( (2, 1),(2, 1) ))(s4)           ## (64 x 64)

    """ Bridge """
    b1 = encoder.get_layer("activation_161").output     ## (30 x 30)
    b1 = ZeroPadding2D((1, 1))(b1)                      ## (32 x 32)

    """ Decoder """
    d1 = decoder_block(b1, s4, 512)                     ## (64 x 64)
    d2 = decoder_block(d1, s3, 256)                     ## (128 x 128)
    d3 = decoder_block(d2, s2, 128)                     ## (256 x 256)
    d4 = decoder_block(d3, s1, 64)                      ## (512 x 512)
    
    """ Output """
    # dropout = Dropout(0.3)(d4)
    dropout = Dropout(0.1)(d4)
    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(dropout)

    model = Model(inputs, outputs, name="InceptionResNetV2-UNet")
    return model

In [11]:
keras.optimizers.Adam().learning_rate

2022-09-30 06:15:34.877953: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-30 06:15:34.992747: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-30 06:15:34.993623: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-30 06:15:34.995481: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

<tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.001>

#### Semantic Segmentation

In [ ]:
K.clear_session()

# Define the model architecture
input_img = Input((512, 512, 3), name='Input')

# model = get_unet(input_img, n_filters=16, dropout=0.1)
model = build_inception_resnetv2_unet(input_shape=(512, 512, 3))

# Compile the model
# optimizer = keras.optimizers.Adam(learning_rate=0.01)
optimizer = keras.optimizers.Adam()
# optimizer = keras.optimizers.Adam(learning_rate=0.0001)
# optimizer = keras.optimizers.RMSprop()
# optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
# optimizer = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)
# optimizer = keras.optimizers.SGD(learning_rate=0.0001, momentum=0.9)

# Class Weight for Imbalance Dataset
class_weight_0 = (cnt0 + cnt1) / cnt0
class_weight_1 = (cnt0 + cnt1) / cnt1
model.compile(optimizer=optimizer,
#               loss=create_weighted_binary_crossentropy(class_weight_0, class_weight_1),
              loss = 'binary_crossentropy',
              metrics=[f1_m, 'accuracy'])

history = model.fit(
  x=x_train,
  y=y_train,
  batch_size=8,
#   epochs=20,
  epochs=50,
  validation_data=(x_val, y_val),
  verbose=1
)

# Generate generalization metrics
scores_training = model.evaluate(x_train, y_train, verbose=0)
print('----------------------------------------------------------------------------')
print("Training Data")
print("F1-Score:", scores_training[1] * 100)
print("Accuracy:", scores_training[2] * 100)

scores_validation = model.evaluate(x_val, y_val, verbose=0)
print('----------------------------------------------------------------------------')
print("Validation Data")
print("F1-Score:", scores_validation[1] * 100)
print("Accuracy:", scores_validation[2] * 100)
print('----------------------------------------------------------------------------')

Epoch 1/50
21/21 [==============================] - 37s 1s/step - loss: 0.5285 - f1_m: 0.4452 - accuracy: 0.8002 - val_loss: 13.5806 - val_f1_m: 0.1942 - val_accuracy: 0.1082
Epoch 2/50
21/21 [==============================] - 20s 974ms/step - loss: 0.2709 - f1_m: 0.5155 - accuracy: 0.9283 - val_loss: 12.5767 - val_f1_m: 0.1960 - val_accuracy: 0.1186
Epoch 3/50
21/21 [==============================] - 20s 968ms/step - loss: 0.2388 - f1_m: 0.5091 - accuracy: 0.9253 - val_loss: 12.6916 - val_f1_m: 0.1976 - val_accuracy: 0.1286
Epoch 4/50
21/21 [==============================] - 20s 971ms/step - loss: 0.2047 - f1_m: 0.5704 - accuracy: 0.9331 - val_loss: 12.0191 - val_f1_m: 0.1975 - val_accuracy: 0.1269
Epoch 5/50
21/21 [==============================] - 20s 973ms/step - loss: 0.1797 - f1_m: 0.6597 - accuracy: 0.9386 - val_loss: 8.2242 - val_f1_m: 0.2073 - val_accuracy: 0.1824
Epoch 6/50
21/21 [==============================] - 20s 974ms/step - loss: 0.1589 - f1_m: 0.7025 - accuracy: 0.942

#### Prediction Using Model and Visualize

In [ ]:
pred = model.predict(x_testing)
arr_hasil = []
image_hasil = []

sz = x_testing.shape[0]
for i in range(sz):
  temp_hasil = []
  temp_image = []
  for j in range(512):
    temp_hasil2 = []
    temp_image2 = []
    for k in range(512):
      if pred[i][j][k] >= 0.5:
        hasil = 1
        image = [0, 0, 0]
      else:
        hasil = 0
        image = [255, 255, 255]
      temp_hasil2.append(hasil)
      temp_image2.append(image)
    temp_hasil.append(temp_hasil2)
    temp_image.append(temp_image2)
  arr_hasil.append(temp_hasil)
  image_hasil.append(temp_image)

arr_hasil = np.array(arr_hasil)
image_hasil = np.array(image_hasil)

print("Testing Data F1-Score: ", end="")
print(sklearn.metrics.f1_score(arr_label.flatten(), arr_hasil.flatten(), average="macro") * 100)
x_testing = x_testing.astype('int32')

In [ ]:
input_concat = np.concatenate((x_testing[17], x_testing[27], x_testing[35]), axis=1)
label_concat = np.concatenate((image_label[17], image_label[27], image_label[35]), axis=1)
hasil_concat = np.concatenate((image_hasil[17], image_hasil[27], image_hasil[35]), axis=1)

print("Input")
plt.figure(figsize=(15, 15))
plt.imshow(input_concat)
plt.axis("off")
plt.savefig('input.png')
plt.show()
print("Label")
plt.figure(figsize=(15, 15))
plt.imshow(label_concat)
plt.axis("off")
plt.savefig('label.png')
plt.show()
print("Hasil/Output")
plt.figure(figsize=(15, 15))
plt.imshow(hasil_concat)
plt.axis("off")
plt.savefig('hasil.png')
plt.show()

#### Transfer Learning Practice

In [ ]:
# K.clear_session()

# base_model = keras.applications.Xception(weights='imagenet', input_shape=(512, 512, 3), include_top=False)
# base_model.trainable = False

# x = base_model.output
# # x = keras.layers.GlobalAveragePooling2D()(x)
# x = pix2pix.upsample(512, 3)(x)
# x = pix2pix.upsample(256, 3)(x)
# x = pix2pix.upsample(128, 3)(x)
# x = pix2pix.upsample(64, 3)(x)
# x = pix2pix.upsample(32, 3)(x)
# outputs = keras.layers.Conv2D(1, 1, activation='sigmoid', padding='same')(x)
# model = keras.Model(base_model.input, outputs)

# print(model.summary)

# # Compile the model
# optimizer = keras.optimizers.Adam()
# # optimizer = keras.optimizers.Adam(learning_rate=0.001)
# # optimizer = keras.optimizers.Adam(learning_rate=0.0001)

# # Class Weight for Imbalance Dataset
# # class_weight_0 = (cnt0 + cnt1) / cnt0
# # class_weight_1 = (cnt0 + cnt1) / cnt1
# model.compile(optimizer=optimizer,
#               loss = 'binary_crossentropy',
#               metrics=[f1_m, 'accuracy'])

# # y_train2 = y_train.reshape(y_train.shape[0], y_train.shape[1] * y_train.shape[2])
# # y_val2 = y_val.reshape(y_val.shape[0], y_val.shape[1] * y_val.shape[2])

# history = model.fit(
#   x=x_train,
#   y=y_train,
#   batch_size=8,
#   epochs=50,
#   validation_data=(x_val, y_val),
#   verbose=1
# )

# # Generate generalization metrics
# scores_training = model.evaluate(x_train, y_train, verbose=0)
# print('----------------------------------------------------------------------------')
# print("Training Data")
# print("F1-Score:", scores_training[1] * 100)
# print("Accuracy:", scores_training[2] * 100)

# scores_validation = model.evaluate(x_val, y_val, verbose=0)
# print('----------------------------------------------------------------------------')
# print("Validation Data")
# print("F1-Score:", scores_validation[1] * 100)
# print("Accuracy:", scores_validation[2] * 100)
# print('----------------------------------------------------------------------------')

In [ ]:
# pred = model.predict(x_testing)
# # pred = pred.reshape(y_testing.shape[0], y_testing.shape[1], y_testing.shape[2])

# arr_hasil = []
# image_hasil = []
# for i in range(16):
#   temp_hasil = []
#   temp_image = []
#   for j in range(512):
#     temp_hasil2 = []
#     temp_image2 = []
#     for k in range(512):
#       if pred[i][j][k] >= 0.5:
#         hasil = 1
#         image = [0, 0, 0]
#       else:
#         hasil = 0
#         image = [255, 255, 255]
#       temp_hasil2.append(hasil)
#       temp_image2.append(image)
#     temp_hasil.append(temp_hasil2)
#     temp_image.append(temp_image2)
#   arr_hasil.append(temp_hasil)
#   image_hasil.append(temp_image)

# arr_hasil = np.array(arr_hasil)
# image_hasil = np.array(image_hasil)

# print("Testing Data F1-Score: ", end="")
# print(sklearn.metrics.f1_score(arr_label.flatten(), arr_hasil.flatten(), average="macro") * 100)
# x_testing = x_testing.astype('int32')

In [ ]:
# input_concat = np.concatenate((x_testing[6], x_testing[9], x_testing[15]), axis=1)
# label_concat = np.concatenate((image_label[6], image_label[9], image_label[15]), axis=1)
# hasil_concat = np.concatenate((image_hasil[6], image_hasil[9], image_hasil[15]), axis=1)

# print("Input")
# plt.figure(figsize=(15,15))
# plt.imshow(input_concat)
# plt.axis("off")
# plt.savefig('input.png')
# plt.show()
# print("Label")
# plt.figure(figsize=(15,15))
# plt.imshow(label_concat)
# plt.axis("off")
# plt.savefig('label.png')
# plt.show()
# print("Hasil/Output")
# plt.figure(figsize=(15,15))
# plt.imshow(hasil_concat)
# plt.axis("off")
# plt.savefig('hasil.png')
# plt.show()